In [7]:
import pandas as pd
import numpy as np1
import re
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool
import warnings
warnings.filterwarnings("ignore")



In [8]:
def preprocess_text(text: str):
    text = re.sub("\€+", " евро ", text)
    text = re.sub("\@\w+", " <tag> ", text)
    text = re.sub("\+\d+", " <phone_number> ", text)
    text = re.sub("\[.*?\]\(.*?\)", "", text)
    text = re.sub("[\*\!\.\,]+", " ", text)
    text = re.sub("\d+", " 1 ", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [9]:
raw_df = pd.read_csv('labeled_data_corpus.csv')
train_df = raw_df.loc[raw_df['subset'] == 'train']
test_df = raw_df.loc[raw_df['subset'] == 'test']
train_df['text'] = train_df['msg'].apply(preprocess_text)
test_df['text'] = test_df['msg'].apply(preprocess_text)

In [10]:
train_df['msg'].apply(preprocess_text).sample(n=20).head(20)

772     важно: если ты не бот и не спамер пройди прове...
1011    важно: если ты не бот и не спамер пройди прове...
3182     hi my name is ekaterina me my husband and my ...
5133                              you were banned (spam) 
3587    ищу 1 бедрум квартиру в пафосе до 1 евро можно...
1518    здравствуйте ищу квартиру в пафосе семья из дв...
2849    #аренда корал бэй/coral bay (пафос) 1 евро 1 с...
3175     привет это бесплатная группа по аренде и прод...
811     добрый день ищу однокомнатную квартиру в ларна...
2297     аренда | 1 -сп квартира | 1 м 1 | agios tycho...
1045    аренда 1 -х комнатной квартиры лимассол район ...
4207    ищу 1 сп квартиру для долгосрочной аренды ( ми...
2278    wao friend you also trade with this amazing co...
3570    ларнака терсефану аренда квартиры 1 спальни бе...
5128    продажа собственник локация: лимассол район li...
1955    продам землю под застройку 🪩 собственник 🪩 пар...
4427    добрый день ищу квартиру в лимассоле на долгий...
4365          

In [11]:
feature_names = ['text']
train_pool = Pool(
    train_df[feature_names], 
    train_df['label'], 
    text_features=["text"], 
    feature_names=feature_names
)

val_pool = Pool(
    test_df[feature_names], 
    test_df['label'], 
    text_features=["text"], 
    feature_names=feature_names
)


In [12]:
args = {
    "iterations" : 1000,
    "learning_rate" : 0.01,
    "loss_function" : 'Logloss',
    "eval_metric" : "F1",
    "verbose" : 50,
    "random_seed" : 42,
}
model = CatBoostClassifier(**args)

model.fit(train_pool, eval_set=val_pool)
preds_class = model.predict(val_pool, prediction_type='Class')
print("Final F1 Score", f1_score(test_df['label'], preds_class))

0:	learn: 0.7735164	test: 0.8054146	best: 0.8054146 (0)	total: 309ms	remaining: 5m 9s
50:	learn: 0.7960985	test: 0.8121442	best: 0.8183556 (4)	total: 8.26s	remaining: 2m 33s
100:	learn: 0.8101266	test: 0.8199234	best: 0.8199234 (74)	total: 14.3s	remaining: 2m 6s
150:	learn: 0.8207283	test: 0.8288973	best: 0.8288973 (143)	total: 20.9s	remaining: 1m 57s
200:	learn: 0.8300469	test: 0.8377358	best: 0.8377358 (192)	total: 26.5s	remaining: 1m 45s
250:	learn: 0.8380414	test: 0.8474576	best: 0.8490566 (243)	total: 32.1s	remaining: 1m 35s
300:	learn: 0.8455361	test: 0.8462998	best: 0.8490566 (243)	total: 38.9s	remaining: 1m 30s
350:	learn: 0.8522727	test: 0.8500949	best: 0.8500949 (335)	total: 45s	remaining: 1m 23s
400:	learn: 0.8599905	test: 0.8571429	best: 0.8571429 (394)	total: 51.4s	remaining: 1m 16s
450:	learn: 0.8613485	test: 0.8517110	best: 0.8571429 (394)	total: 57.5s	remaining: 1m 10s
500:	learn: 0.8673324	test: 0.8538899	best: 0.8571429 (394)	total: 1m 3s	remaining: 1m 3s
550:	learn: 

In [13]:
test_df['prediction'] = preds_class
test_df.loc[test_df['prediction'] != test_df['label'], ['text', 'label']].sample(20)

,text,label
5406,сдам порядочной девушке комнату в доме в район...,1
6119,аренда пейя район пафоса в долгосрочную аренду...,0
5929,- 1 adults 1 child 1 years old without animals...,0
5994,__ tommy’s estate agency 1 / 1 __ ️аренда•пота...,0
6231,agents|arto estates ltd | reg 1 | lic 1 sale: ...,0
5423,hi we_re a married couple with no children no ...,1
6239,office for rent office building rental agios n...,0
6535,#аренда пейя район пафоса в долгосрочную аренд...,0
5620,лимасол аренда пентхауса 1 евро готова к засел...,1
6005,#арендапафос #аренда студии и 1 -х спальной кв...,0
